In [22]:
import pandas as pd

#cluster_summary = pd.read_csv("/home/muthu/Desktop/New Folder/notebook/cluster_summary.csv"); cluster_summary.head()
cluster_summary = pl.scan_parquet("/home/muthu/Desktop/New Folder/notebook/cluster_summary.parquet"); cluster_summary

In [23]:
#features_clustered = pd.read_csv("/home/muthu/Desktop/New Folder/notebook/features_clustered.csv"); features_clustered
features_clustered = pl.scan_parquet("/home/muthu/Desktop/New Folder/notebook/features_clustered.parquet"); features_clustered

In [ ]:
from neo4j import GraphDatabase
import polars as pl

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j","12345678"))

with driver.session() as session:
    for row in cluster_summary.iter_rows(named=True):
        session.run(
            "MERGE (:Cluster {cluster_id:$id, size:$size, summary:$summary})",
            id=row["cluster_id"], size=row["cluster_size"], summary=row["summary_text"]
        )

    for cid in features_clustered["cluster_id"].unique():
        sample = features_clustered.filter(pl.col("cluster_id")==cid).head(2)  # sample users
        for r in sample.iter_rows(named=True):
            session.run("""
                MERGE (c:Customer {id:$id})
                WITH c
                MATCH (cl:Cluster {cluster_id:$cid})
                MERGE (c)-[:IN_CLUSTER]->(cl)
            """, id=r["muid"], cid=cid)


AttributeError: 'LazyFrame' object has no attribute 'iter_rows'